# FYP ICT3909 - Yolov8 Training
---

**Name:** Andrea Filiberto Lucas  
**ID No:** 0279704L

---

## Automated Dataset Management with Roboflow and Folder Organization

This script automates the process of managing a dataset using Roboflow. It creates necessary directories, checks for installed dependencies, installs them if missing, and downloads the dataset. It also organizes the dataset into a structured folder hierarchy, ensuring everything is ready for further use.

In [ ]:
import os
import subprocess
import shutil
import importlib

# Constants for colored output
COLORS = {
    "green": "\033[92m",  # Green text
    "red": "\033[91m",    # Red text
    "reset": "\033[0m"    # Reset to default color
}

# Define the path to the Versions folder and the target subfolder
versions_path = os.path.abspath(os.path.join("..", "Other", "Versions"))
target_subfolder = os.path.join(versions_path, "NGD-Yolov")

# Check if the Versions folder exists, if not, create it
if not os.path.exists(versions_path):
    os.makedirs(versions_path)
    print(f"[{COLORS['green']}✔{COLORS['reset']}] Folder created at: {versions_path}")

# Check if the MDD-AFL-Yolov8 subfolder exists
if os.path.exists(target_subfolder):
    print(f"[{COLORS['green']}✔{COLORS['reset']}] The subfolder '{target_subfolder}' already exists. Skipping download!")
else:
    # Check if roboflow is installed
    if importlib.util.find_spec("roboflow") is not None:                                                                                                        # type: ignore
        print(f"[{COLORS['green']}✔{COLORS['reset']}] Roboflow is already installed!")
    else:
        # Install roboflow using pip
        try:
            subprocess.check_call(["pip", "install", "roboflow"])
            print(f"[{COLORS['green']}✔{COLORS['reset']}] Roboflow successfully installed!")
        except subprocess.CalledProcessError as e:
            print(f"[{COLORS['red']}✖{COLORS['reset']}]Failed to install Roboflow. Please check your setup.")
            raise e

    # Import and use Roboflow
    from roboflow import Roboflow                                                                                                                               # type: ignore

    # Prompt the user for their API key
    print("Please enter your Roboflow API key to download the dataset...")
    api_key = input("Please enter your Roboflow API key: ")

    # Initialize Roboflow with the provided API key
    rf = Roboflow(api_key=api_key)

    # Retrieve project and version
    project = rf.workspace("ict3909-fyp").project("news-graphic-dataset")
    version = project.version(1)

    # Download the dataset
    dataset = version.download("yolov8")

    current_folder = os.getcwd()  # Get the current working directory
    original_folder = os.path.join(current_folder, "News-Graphic-Dataset-1")
    renamed_folder = os.path.join(current_folder, "NGD-Yolov8")
    target_folder = os.path.join(versions_path, "NGD-Yolov8")

    # Check if the original folder exists   
    if os.path.exists(original_folder):
        # Rename the folder
        os.rename(original_folder, renamed_folder)

        # Move the renamed folder to ../Versions/
        shutil.move(renamed_folder, target_folder)
        print(f"[{COLORS['green']}✔{COLORS['reset']}] Folder downloaded to: {target_folder}")
    else:
        print(f"[{COLORS['red']}✖{COLORS['reset']}]Folder '{original_folder}' does not exist. No action taken.")


[✔] Roboflow is already installed!
Please enter your Roboflow API key to download the dataset...
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to News-Graphic-Dataset-1 in yolov8:: 100%|██████████| 11412/11412 [00:27<00:00, 407.58it/s]


[✔] Folder downloaded to: d:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov8


## Automated Library Installer in Python

This script automates checking and installing libraries from a JSON file. It verifies installations, installs missing libraries via `pip`, and provides clear, colored output for success or errors. With built-in error handling and preloaded common libraries, it simplifies dependency management in Python projects.

In [ ]:
import json
import importlib.util

# Path to the JSON file
lib_file_path = os.path.join("..", "Other", "Libraries", "Yolo.json")

# Read the libraries from the JSON file
try:
    with open(lib_file_path, 'r') as file:
        libraries = json.load(file)
except FileNotFoundError:
    print(f"{COLORS['red']}Error: Library file not found at {lib_file_path}{COLORS['reset']}")
    exit(1)
except json.JSONDecodeError:
    print(f"{COLORS['red']}Error: Failed to decode JSON from the library file.{COLORS['reset']}")
    exit(1)

# Function to check and install libraries
def check_and_install_libraries(libraries):
    for lib, import_name in libraries.items():
        # Check if the library is installed by checking its module spec
        if importlib.util.find_spec(import_name) is not None:
            print(f"[{COLORS['green']}✔{COLORS['reset']}] Library '{lib}' is already installed.")
        else:
            # If the library is not found, try to install it
            print(f"[{COLORS['red']}✖{COLORS['reset']}] Library '{lib}' is not installed. Installing...")
            try:
                subprocess.check_call(["pip", "install", lib])
                print(f"[{COLORS['green']}✔{COLORS['reset']}] Successfully installed '{lib}'.")
            except subprocess.CalledProcessError:
                print(f"[{COLORS['red']}✖{COLORS['reset']}] Failed to install '{lib}'. Please install it manually.")


# Execute the function to check and install libraries
check_and_install_libraries(libraries)

# Import necessary libraries 
import time
import torch                                                                                                                                                    #type: ignore
import matplotlib.patches as patches                                                                                                                            #type: ignore
from ultralytics import YOLO                                                                                                                                    #type: ignore

[✔] Library 'opencv-python' is already installed.
[✔] Library 'matplotlib' is already installed.
[✔] Library 'tqdm' is already installed.
[✔] Library 'ultralytics' is already installed.


## Determine the Available Device for Computation

This function identifies and selects the best available device for running deep learning models, prioritizing hardware acceleration via CUDA (GPU) or MPS (Metal Performance Shaders). If no hardware acceleration is available, it defaults to the CPU and notifies the user.

In [3]:
# Determine the available device: CUDA, MPS, or CPU.
def determine_device():
    if torch.cuda.is_available():
        device = "cuda"
        print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    elif torch.backends.mps.is_available():
        device = "mps"
        print("Using MPS (Metal Performance Shaders) for acceleration.")
    else:
        device = "cpu"
        print("Using CPU. Consider enabling GPU or MPS for faster training.")
    return device

## Training process for Yolov object detector.
### Experiment Configuration for YOLOv8

Defines the path to the `data.yaml` file and key hyperparameters for conducting experiments with YOLOv8. These settings include the number of epochs, image size, batch size, and a customizable experiment name for version tracking and organization.

In [4]:
# Path to data.yaml
data_yaml_path = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)), "Other", "Versions", "NGD-Yolov8", "data.yaml")

# Prompt user for hyperparameters
try:
    epochs = int(input("Enter the number of epochs: "))
    experiment_number = int(input("Enter the experiment number: "))
    experiment_name = f"NGD-Yolov8_v{experiment_number}"
except ValueError:
    print("Invalid input. Please enter valid numbers for epochs and experiment number.")
    raise

# Display selected parameters
print(f"Selected epochs: {epochs}")
print(f"Experiment name: {experiment_name}")

# Hyperparameters
imgsz = 640
batch_size = 8

Selected epochs: 200
Experiment name: NGD-Yolov8_v5


In [6]:
# Timer start
start_time = time.time()

# Create or load a YOLOv8 model (from scratch)
try:
    model = YOLO("yolov8m.yaml")
except FileNotFoundError:
    print("Error: YOLOv8 configuration file 'yolo8.yaml' not found. Check setup!")
    raise

# Train
try:
    model.train(
        data=data_yaml_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch_size,
        name=experiment_name,
        pretrained=False,  
        plots=True,
        device = "cuda" if torch.cuda.is_available() else "cpu",
        patience=10,
        dropout=0.4,
        cos_lr=True,
        lr0=0.0001,
        lrf=0.00001,
    )
except Exception as e:
    print(f"Error during training: {e}")
    raise

# Timer end
end_time = time.time()
# Calculate elapsed time
elapsed_time = end_time - start_time
# Convert seconds to hours, minutes, and seconds
hours = int(elapsed_time // 3600)
minutes = int((elapsed_time % 3600) // 60)
seconds = int(elapsed_time % 60)
print(f"Training completed in {hours}h {minutes}m {seconds}s.")

# Evaluate
try:
    metrics = model.val()
    print("Validation Metrics:", metrics)
except Exception as e:
    print(f"Error during validation: {e}")
    raise

New https://pypi.org/project/ultralytics/8.3.103 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.101  Python-3.12.7 torch-2.8.0.dev20250405+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.yaml, data=d:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov8\data.yaml, epochs=200, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=NGD-Yolov8_v5, exist_ok=False, pretrained=False, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.4, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=F

train: Scanning D:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov8\train\labels... 5250 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5250/5250 [00:10<00:00, 486.85it/s]


train: New cache created: D:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov8\train\labels.cache


val: Scanning D:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov8\valid\labels... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<00:00, 558.31it/s]

val: New cache created: D:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov8\valid\labels.cache


...


57 epochs completed in 0.830 hours.
Optimizer stripped from runs\detect\NGD-Yolov8_v5\weights\last.pt, 52.0MB
Optimizer stripped from runs\detect\NGD-Yolov8_v5\weights\best.pt, 52.0MB

Validating runs\detect\NGD-Yolov8_v5\weights\best.pt...
Ultralytics 8.3.101  Python-3.12.7 torch-2.8.0.dev20250405+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLOv8m summary (fused): 92 layers, 25,843,234 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:01<00:00, 10.17it/s]


                   all        300        943      0.927      0.871      0.937      0.752
 Breaking News Graphic         47         48      0.956      0.895       0.95      0.725
Digital On-Screen Graphic        289        392      0.961      0.926      0.967      0.822
   Lower Third Graphic        147        156      0.952      0.884      0.955      0.769
         News Headline         99        100      0.971      0.989      0.978      0.955
           News Ticker        125        126       0.87      0.905       0.96      0.612
    Other News Graphic         92        121      0.854      0.628      0.813      0.628
Speed: 0.1ms preprocess, 1.9ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\NGD-Yolov8_v5
Training completed in 0h 50m 57s.
Ultralytics 8.3.101  Python-3.12.7 torch-2.8.0.dev20250405+cu128 CUDA:0 (NVIDIA GeForce RTX 4090, 24564MiB)
YOLOv8m summary (fused): 92 layers, 25,843,234 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning D:\FYP\Accurate-Name-Extraction\Other\Versions\NGD-Yolov8\valid\labels.cache... 300 images, 0 backgrounds, 0 corrupt: 100%|██████████| 300/300 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:02<00:00, 16.90it/s]


                   all        300        943      0.926      0.869      0.937      0.752
 Breaking News Graphic         47         48      0.955      0.893       0.95      0.723
Digital On-Screen Graphic        289        392      0.962      0.926      0.967      0.821
   Lower Third Graphic        147        156      0.952      0.881      0.955       0.77
         News Headline         99        100      0.971      0.989      0.978      0.954
           News Ticker        125        126      0.856      0.898      0.958       0.61
    Other News Graphic         92        121      0.862      0.628      0.813      0.631
Speed: 0.2ms preprocess, 3.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs\detect\NGD-Yolov8_v52
Validation Metrics: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002

## Results

### Ultralytics Results 

This script facilitates the selection and evaluation of YOLOv8 detection results stored in the `runs/detect` directory. It identifies folders ending with "R," lets the user choose one, and displays evaluation images within the selected folder (excluding "val_batch" files). Additionally, it checks the corresponding "non-R" folder for a `results.png` file and displays it if found.

In [ ]:
# Path to the "runs/detect" folder
detect_folder_path = os.path.join(os.getcwd(), "runs", "detect")

# Get all folders ending with "R" and sort them in ascending order
folders_ending_with_R = sorted([
    folder for folder in os.listdir(detect_folder_path)
    if folder.endswith("R") 
    and os.path.isdir(os.path.join(detect_folder_path, folder))
])

# Check if there are any folders to choose from
if not folders_ending_with_R:
    print("No Results folders ending with 'R' found!")
else:
    # Display available folders for user selection
    print("Available folders:")
    for i, folder in enumerate(folders_ending_with_R, start=1):
        print(f"{i}. {folder}")
    
    # Prompt the user to select a folder
    try:
        choice = int(input("Enter the number corresponding to the folder you want to use: ")) - 1
        if choice < 0 or choice >= len(folders_ending_with_R):
            raise IndexError
        selected_folder = folders_ending_with_R[choice]
    except (ValueError, IndexError):
        print("Invalid selection. Please restart and try again.")
        raise
    
    print(f"\nSelected folder: {selected_folder}")
    # Path to the selected folder
    eval_folder_path = os.path.join(detect_folder_path, selected_folder)

    # List of evaluation images excluding "val_batch" files
    evaluation_images = [
        img for img in os.listdir(eval_folder_path)
        if img.lower().endswith(('.png', '.jpg')) and "val_batch" not in img
    ]

    # Display images
    if not evaluation_images:
        print("No evaluation images found in the selected folder.")
    else:
        print("\nDisplaying evaluation images:")
        for img_file in evaluation_images:
            img_path = os.path.join(eval_folder_path, img_file)
            display(Image(filename=img_path))

    # Path to the non-R version of the selected folder
    non_r_folder = selected_folder.rstrip("R")
    non_r_folder_path = os.path.join(detect_folder_path, non_r_folder)

    # Check and display "results.png" if it exists
    results_png_path = os.path.join(non_r_folder_path, "results.png")
    if os.path.exists(results_png_path):
        print("\nDisplaying results.png from the non-R version of the folder:")
        display(Image(filename=results_png_path))
    else:
        print("No results.png found in the non-R version of the selected folder.")